# Store Sales: Data Visualization 

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

c:\Users\Cyril\anaconda3\envs\DataScience3_9\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%run -n main.py

In [3]:
df = pd.read_csv("../datasets/train.csv", index_col="date", parse_dates=True)
print(df.index.dtype)
df.head()

datetime64[ns]


,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [75]:
weekly = df.drop(["store_nbr", "id"], axis=1).resample("W").sum().reset_index()

group_sales = weekly.groupby(weekly.index)["sales"].sum()
monthly_rolling = group_sales.rolling(12, center=True)
yearly_rolling = group_sales.rolling(48, center=True)
data = pd.DataFrame({"date":weekly.index,
                     "sales":weekly["sales"],
                     "monthly_rolling_mean":monthly_rolling.mean(),
                     "yearly_rolling_mean":yearly_rolling.mean()})
px.line(data, x="date", y=["sales", "monthly_rolling_mean", "yearly_rolling_mean"], title="Sales over time").show()

In [67]:
weekly_type = df.drop(["store_nbr", "id"], axis=1).groupby("family").resample("W").sum().reset_index()
px.line(weekly_type, x="date", y="sales", color="family", title="Sales by Product").show()

In [66]:
from scipy.stats import zscore
df["sales_zscore"] = df.groupby("family")["sales"].transform(lambda x : zscore(x,ddof=1))
norm_weekly_type = df.groupby("family")["sales_zscore"].resample("W").mean().reset_index()
px.line(norm_weekly_type, x="date", y="sales_zscore", color="family", title="Sales Z-Scores by Product").show()

In [68]:
by_weekday = df.groupby(df.index.dayofweek).mean()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
fig = px.bar(by_weekday, x=by_weekday.index, y="sales", title="Sales by Weekday")
fig.show()